RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
module 'keras' has no attribute 'utils'

In [3]:

# Constants
MAX_SEQ_LEN = 100
LSTM_UNITS = 64

# Task-specific constants
NUM_EMOTION_CLASSES = 7
NUM_SENTIMENT_CLASSES = 3
NUM_TOXICITY_CLASSES = 7

# Initialize the BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Input and BERT layers
input_layer = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name='input')
bert_output = bert_model(input_layer)[0]  # Use index 0 to get the last-layer hidden states

# Bidirectional LSTM layer
bi_lstm_layer = Bidirectional(LSTM(LSTM_UNITS))(bert_output)

# Task-specific layers
emotion_output = Dense(NUM_EMOTION_CLASSES, activation='softmax', name='emotion_output')(bi_lstm_layer)
sentiment_output = Dense(NUM_SENTIMENT_CLASSES, activation='softmax', name='sentiment_output')(bi_lstm_layer)
toxicity_input = Concatenate(name="concatenation_of_blstm_emotion")([bi_lstm_layer, emotion_output])
toxicity_output = Dense(NUM_TOXICITY_CLASSES, activation='softmax', name='toxicity_output')(toxicity_input)

# Define the model
model = Model(inputs=input_layer, outputs=[emotion_output, sentiment_output, toxicity_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'emotion_output': 'categorical_crossentropy',
        'sentiment_output': 'categorical_crossentropy',
        'toxicity_output': 'categorical_crossentropy'
    },
    metrics=[AUC(),{'emotion_output': 'accuracy',
             'sentiment_output': 'accuracy',
             'toxicity_output': 'accuracy'}])

# Summary of the model
model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 100)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input[0][0]']                  
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 100,                                               
                                 768),                                                            
                                 pooler_output=(Non                                           

In [10]:
# Sample data
data = {
    'text': [
        "I'm feeling happy today!",
        "This movie is amazing and uplifting.",
        "The weather is gloomy and sad.",
        "The restaurant service was terrible.",
        "I feel neutral about this book.",
        "The speech was inspiring and motivational.",
        "The internet trolls are spreading toxicity."
    ],
    'emotion': ['joy', 'joy', 'sadness', 'anger', 'neutral', 'joy', 'anger'],
    'sentiment': ['positive', 'positive', 'negative', 'negative', 'neutral', 'positive', 'negative'],
    'toxicity': [0, 0, 1, 1, 0, 0, 1]
}

emotions = ['joy', 'sadness','anger','neutral','surprise','fear']
sentiments = ['positive','neutral','negative']
# Create a DataFrame
df = pd.DataFrame(data)

# Tokenize and encode the texts
encoded_inputs = bert_tokenizer(df['text'].tolist(), padding='max_length', truncation=True,
                                max_length=MAX_SEQ_LEN, return_tensors='tf')
# Convert emotion labels to one-hot encoded vectors
emotion_labels = np.array([emotions.index(label) for label in df['emotion']])
emotion_labels = to_categorical(emotion_labels, num_classes=NUM_EMOTION_CLASSES)

# Convert sentiment labels to one-hot encoded vectors
sentiment_labels = np.array([sentiments.index(label) for label in df['sentiment']])
sentiment_labels = to_categorical(sentiment_labels, num_classes=NUM_SENTIMENT_CLASSES)

# Convert toxicity labels to one-hot encoded vectors
toxicity_labels = to_categorical(df['toxicity'], num_classes=NUM_TOXICITY_CLASSES)

# Create the sample dataset
dataset = tf.data.Dataset.from_tensor_slices((encoded_inputs['input_ids'],
                                              encoded_inputs['attention_mask'],
                                              emotion_labels,
                                              sentiment_labels,
                                              toxicity_labels))
dataset = dataset.shuffle(len(df)).batch(1)  # Adjust batch size as needed

# Print a sample batch from the dataset
for batch in dataset.take(1):
    input_ids, attention_mask, emotion_labels, sentiment_labels, toxicity_labels = batch
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    print("Emotion Labels:", emotion_labels)
    print("Sentiment Labels:", sentiment_labels)
    print("Toxicity Labels:", toxicity_labels)

Input IDs: tf.Tensor(
[[  101  2023  3185  2003  6429  1998  2039 26644  1012   102     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]], shape=(1, 100), dtype=int32)
Attention Mask: tf.Tensor(
[[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 100), dtype=int32)
Emotion Labels: tf.Tensor([[1. 0. 0. 0. 0. 0. 0.]], shape=(1, 7

In [11]:
# for epoch in range(10):
#     for batch in dataset:
#         input_ids, attention_mask, emotion_labels, sentiment_labels, toxicity_labels = batch
#         model.train_on_batch([input_ids, attention_mask], [emotion_labels, sentiment_labels, toxicity_labels])
model.fit(dataset, epochs=10)


Epoch 1/10


ValueError: in user code:

    File "e:\Mini Projects\Thesis\report-system-back\.venv\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "e:\Mini Projects\Thesis\report-system-back\.venv\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "e:\Mini Projects\Thesis\report-system-back\.venv\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "e:\Mini Projects\Thesis\report-system-back\.venv\Lib\site-packages\keras\engine\training.py", line 1047, in train_step
        x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(data)
    File "e:\Mini Projects\Thesis\report-system-back\.venv\Lib\site-packages\keras\engine\data_adapter.py", line 1775, in unpack_x_y_sample_weight
        raise ValueError(error_msg)

    ValueError: Data is e

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.metrics import AUC

ModuleNotFoundError: No module named 'sklearn'